In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toxicity-detection/val.csv
/kaggle/input/toxicity-detection/train.csv
/kaggle/input/toxicity-detection/test.csv


In [21]:
import pandas as pd
from datasets import Dataset

# Load your data
train_df = pd.read_csv("/kaggle/input/toxicity-detection/train.csv")
val_df = pd.read_csv("/kaggle/input/toxicity-detection/val.csv")

# Rename target column for Trainer
train_df = train_df.rename(columns={"toxic": "label"})
val_df = val_df.rename(columns={"toxic": "label"})

# Convert to Hugging Face datasets
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize(batch):
    return tokenizer(batch["comment_text"], padding="max_length", truncation=True, max_length=512)

train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
val_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/125696 [00:00<?, ? examples/s]

Map:   0%|          | 0/26935 [00:00<?, ? examples/s]

In [12]:
from transformers import RobertaForSequenceClassification
from torch.nn import BCEWithLogitsLoss
import torch

# Compute class weights (inverse frequency)
num_pos = train_df["label"].sum()
num_neg = len(train_df) - num_pos
weight = torch.tensor([num_neg / num_pos], dtype=torch.float)

# Custom model with weighted loss
class RobertaForToxicClassification(RobertaForSequenceClassification):
    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        kwargs.pop("num_items_in_batch", None)
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=None, **kwargs)
        logits = outputs.logits

        if labels is not None:
            loss_fct = BCEWithLogitsLoss(pos_weight=weight.to(logits.device))
            labels = labels.unsqueeze(1).float()  # shape: [batch_size, 1]
            loss = loss_fct(logits, labels)
            return {"loss": loss, "logits": logits}
        else:
            return {"logits": logits}

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def compute_metrics(p):
    preds = torch.sigmoid(torch.tensor(p.predictions)).numpy() > 0.5
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "precision": precision_score(labels, preds, zero_division=0),
        "recall": recall_score(labels, preds, zero_division=0),
        "f1": f1_score(labels, preds, zero_division=0)
    }


In [18]:
from transformers import TrainingArguments, Trainer

model = RobertaForToxicClassification.from_pretrained("roberta-base", num_labels=1)
train_ds_small = train_ds.shuffle(seed=42).select(range(len(train_ds) // 5))

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,         
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2, 
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,                              
    report_to="none"                        
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds_small,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics
)

trainer.train()

Some weights of RobertaForToxicClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.127200,0.543705,0.964210,0.819318,0.816226,0.817769
2,0.906200,0.588877,0.963542,0.799139,0.840755,0.819419
3,0.656100,0.735586,0.964878,0.808249,0.843019,0.825268


TrainOutput(global_step=2358, training_loss=0.896513624247906, metrics={'train_runtime': 4956.8443, 'train_samples_per_second': 15.215, 'train_steps_per_second': 0.476, 'total_flos': 1.9842868299377664e+16, 'train_loss': 0.896513624247906, 'epoch': 3.0})

In [22]:
test_df = pd.read_csv("/kaggle/input/toxicity-detection/test.csv")
test_df = test_df.rename(columns={"toxic": "label"})
test_ds = Dataset.from_pandas(test_df)
test_ds = test_ds.map(tokenize, batched=True)
test_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/26936 [00:00<?, ? examples/s]

In [24]:
results = trainer.evaluate(eval_dataset=test_ds)
print(results)

{'eval_loss': 0.7270007729530334, 'eval_accuracy': 0.9652138402138403, 'eval_precision': 0.8099891422366993, 'eval_recall': 0.8445283018867924, 'eval_f1': 0.8268982080177352, 'eval_runtime': 416.5595, 'eval_samples_per_second': 64.663, 'eval_steps_per_second': 4.043, 'epoch': 3.0}


In [25]:
trainer.save_model("./toxic_roberta_model")  # Save model weights + config
tokenizer.save_pretrained("./toxic_roberta_model")  # Save tokenizer files

('./toxic_roberta_model/tokenizer_config.json',
 './toxic_roberta_model/special_tokens_map.json',
 './toxic_roberta_model/vocab.json',
 './toxic_roberta_model/merges.txt',
 './toxic_roberta_model/added_tokens.json',
 './toxic_roberta_model/tokenizer.json')

In [26]:
!zip -r toxic_roberta_model.zip ./toxic_roberta_model

  adding: toxic_roberta_model/ (stored 0%)
  adding: toxic_roberta_model/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 8%)
  adding: toxic_roberta_model/tokenizer.json (deflated 82%)
  adding: toxic_roberta_model/vocab.json (deflated 59%)
  adding: toxic_roberta_model/tokenizer_config.json (deflated 75%)
  adding: toxic_roberta_model/training_args.bin (deflated 51%)
  adding: toxic_roberta_model/special_tokens_map.json (deflated 52%)
  adding: toxic_roberta_model/merges.txt (deflated 53%)
  adding: toxic_roberta_model/config.json (deflated 50%)
